# Visor Finance Capital Momentum Gradient

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas.util.testing as tm
import requests
%matplotlib inline
import datetime as dt
import seaborn as sns
import numpy as np
from matplotlib import rc
import os
plt.rcParams.update({'figure.max_open_warning': 0})
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Getting Uniswap Pool Data 

In [ ]:
#bitquery api 
#https://bitquery.io/

Bitquery to CSV

To add a new csv file

1. name_of_dataframe = pd.read_csv("name_of_file.csv")
2. name_of_dataframe.shape to see how big it is 
3. name of dataframe.summary 
4. name_of_dataframe.head() to check it out 
5. add new columns by doing like name_of_dataframe['new_column'] = set to equation or multiply to columns etc.
6. if you want to remove a column just do del name_of_dataframe['new_column']  and new_column column will be deleted

### Current amount in top Uniswap pools based on 7 day volume

In [ ]:
total_in_pools =2654014452
visor_total = total_in_pools*.05
visor_total

### 15 Top Uniswap Pairs - upper and lower bounds charted w/ moving average (custom window function)

### USDC-ETH

In [ ]:
usdc_eth = pd.read_csv("../input/weth-usdc/WETH_USDC.csv")
#if you want to see the csv just run below:
#usdc_eth.head()

In [ ]:
#if you need to delete a column example
#del usdc_eth['Unnamed: 0']

In [ ]:
#calculating 7 day twap and delta is for it to be hourly
close_price = usdc_eth["close_price"].astype(dtype=float)
delta = int(7*24)
usdc_eth["twap_7d"] = (close_price.cumsum() - close_price.shift(delta).cumsum())/(delta)

In [ ]:
total_usdc_eth = usdc_eth['quoteAmount'].sum()

In [ ]:
#converting dates to datetime and then making that the index below
usdc_eth['time'] = pd.to_datetime(usdc_eth['timeInterval.minute'], format = '%Y-%m-%d')

In [ ]:
usdc_eth.index = usdc_eth["time"]

In [ ]:
#set the time period because this is when visor started on coingecko 
usdc_eth = usdc_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
#rename columns for example or if you just want to see the columns do usdc_eth.columns
usdc_eth.columns 

**the graph call**
{
 pair(id: "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc"){
     token0 {
       id
       symbol
       name
       derivedETH
     }
     token1 {
       id
       symbol
       name
       derivedETH
     }
     reserve0
     reserve1
     reserveUSD
     trackedReserveETH
     token0Price
     token1Price
     volumeUSD
     txCount
 }
}

In [ ]:
usdc_total = usdc_eth['total_fees_estimate_(USDC)'].sum()

In [ ]:
usdc_eth['fees_eth']=usdc_eth['total_fees_estimate_(USDC)']*0.00036108464909277

In [ ]:
usdc_eth["visor_fees"] = usdc_eth['fees_eth']*.05
visor_fees_usdc_eth = usdc_eth['visor_fees'].sum()

In [ ]:
usdc_eth.head()

In [ ]:
#graphing basic up/down patterns, overall trends, and anomalies 
#blue is hourly data and the orange is daily 
y = usdc_eth['close_price']
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(y,marker='.', linestyle='-', linewidth=0.5, label='Hourly')
#change the d to M if you want to see monthly for example 
ax.plot(y.resample('d').mean(),marker='o', markersize=8, linestyle='-', label='Daily Mean Resample')
ax.set_ylabel('Close Price')
ax.set_xlabel('Time')
plt.title('USDC ETH Price',fontsize=20)
ax.legend();

In [ ]:
def moving_average(series, n):
    return np.average(series[-n:])
#change 24 if you want to see a different amount of time 
moving_average(usdc_eth['close_price'], 24)
# prediction for the last observed day (past 24 hours)

In [ ]:
def moving_average(series, n):
    return np.average(series[-n:])

moving_average(usdc_eth['close_price'], 3)
# prediction for the last observed day (past 3 hours)

In [ ]:
#nothing needs to be changed here it is just the definition of plotting the moving average 
#and seeing upper and lower bounds 
def plotMovingAverage(series, window, plot_intervals=False, scale=1.96, plot_anomalies=False):
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, "r--", label="Upper Bound / Lower Bound")
        plt.plot(lower_bound, "r--")
        
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            plt.plot(anomalies, "ro", markersize=10)
        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

**Previous 1 Hours**

In [ ]:
#just change the number depending how many hours you want to be plotted 
plotMovingAverage(usdc_eth['close_price'],2, plot_intervals=True) 
plt.title('USDC ETH Moving Average Past 1 Hour',fontsize=15)
#smooth the previous 1 hour

**Previous 8 Hours**

In [ ]:
#just change the number depending how many hours you want to be plotted 
plotMovingAverage(usdc_eth['close_price'],8, plot_intervals=True) 
plt.title('USDC ETH Moving Average Past 8 Hours',fontsize=15)
#smooth the previous 8 hours

**Previous 1 Hours**

In [ ]:
#just change the number depending how many hours you want to be plotted 
plotMovingAverage(usdc_eth['close_price'],2, plot_intervals=True) 
plt.title('USDC ETH Moving Average Past 1 Hours',fontsize=15)
#smooth the previous 1 hours

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = usdc_eth['close_price'].rolling(window).mean()
mae = mean_absolute_error(usdc_eth.close_price[window:], rolling_mean[window:])
deviation = np.std(usdc_eth.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
#lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 

**Lower Bound of Previous Hour**

In [ ]:
lower_bound[1005]

**Lower Bound of Previous 5 Hours**

In [ ]:
lower_bound[1000]

In [ ]:
deviation

In [ ]:
#mean absolute error (very tiny number)
mae

**Upper Bound of Previous Hour**

In [ ]:
upper_bound = rolling_mean + (mae + scale * deviation)
#change like you change the lower_bound if you want to see other datapoints or whole thing
upper_bound[1005]

**Upper Bound of Previous 5 Hours**

In [ ]:
upper_bound[1000]

In [ ]:
#to see total of column called visor_fees
#visor_usdc_eth_fees = usdc_eth['visor_fees'].sum()
#visor_usdc_eth_fees

### USDC-USDT

In [ ]:
usdc_usdt = pd.read_csv('../input/usdc-usdt/USDT_USDC.csv')

In [ ]:
del usdc_usdt['Unnamed: 0']

In [ ]:
usdc_usdt.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(USDC)',
       'Cumulative_fees_estimate_USDC', 'twap_7d']

In [ ]:
usdc_usdt['total_fees'] = usdc_usdt['total_fees_estimate_(USDC)']*0.0003643

In [ ]:
#converting dates to datetime and then making that the index below
usdc_usdt['time'] = pd.to_datetime(usdc_usdt['timeInterval.minute'], format = '%Y-%m-%d')
usdc_usdt.index = usdc_usdt["time"]
#set the time period because this is when visor started on coingecko 
usdc_usdt = usdc_usdt.loc['2021-03-15':'2021-04-25']

In [ ]:
#graphing basic up/down patterns, overall trends, and anomalies 
#blue is hourly data and the orange is daily 
y = usdc_usdt['close_price']
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(y,marker='.', linestyle='-', linewidth=0.5, label='Hourly')
#change the d to M if you want to see monthly for example 
ax.plot(y.resample('d').mean(),marker='o', markersize=8, linestyle='-', label='Daily Mean Resample')
ax.set_ylabel('Close Price')
ax.set_xlabel('Time')
plt.title('USDC USDT Price',fontsize=20)
ax.legend();

In [ ]:
moving_average(usdc_usdt['close_price'], 24)
# prediction for the last observed day (past 24 hours)

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = usdc_usdt['close_price'].rolling(window).mean()
mae = mean_absolute_error(usdc_usdt.close_price[window:], rolling_mean[window:])
deviation = np.std(usdc_usdt.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
#lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 

In [ ]:
lower_bound[1007]

In [ ]:
upper_bound = rolling_mean + (mae + scale * deviation)
#change like you change the lower_bound if you want to see other datapoints or whole thing
upper_bound[1007]

In [ ]:
usdc_usdt['visor_fees']=usdc_usdt['total_fees']*.05

In [ ]:
usdc_usdt_visor_fees = usdc_usdt['visor_fees'].sum()

In [ ]:
usdc_usdt_visor_fees

**Previous 8 Hours**

In [ ]:
plotMovingAverage(usdc_usdt['close_price'],8, plot_intervals=True) 
plt.title('USDC USDT Moving Average Past 8 Hours',fontsize=15)

**Previous 1 Hours**

In [ ]:
plotMovingAverage(usdc_usdt['close_price'],2, plot_intervals=True) 
plt.title('USDC USDT Moving Average Past 1 Hours',fontsize=15)

### RAI-ETH

In [ ]:
rai_eth = pd.read_csv("../input/rai-eth/WETH_RAI.csv")

In [ ]:
del rai_eth['Unnamed: 0']

In [ ]:
rai_eth.columns = ['timeInterval.minute', 'baseCurrency.symbol',
       'baseCurrency.address', 'baseAmount', 'quoteCurrency.symbol',
       'quoteCurrency.address', 'quoteAmount', 'trades', 'quotePrice',
       'maximum_price', 'minimum_price', 'open_price', 'close_price',
       'total_fees_estimate_(RAI)', 'Cum_fees_estimate_RAI', 'twap_7d']

In [ ]:
rai_eth['fees_estimate_eth']=rai_eth['total_fees_estimate_(RAI)']*0.00108492

In [ ]:
rai_eth['time'] = pd.to_datetime(rai_eth['timeInterval.minute'], format = '%Y-%m-%d')
rai_eth.index =rai_eth["time"]
rai_eth= rai_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
#graphing basic up/down patterns, overall trends, and anomalies 
#blue is hourly data and the orange is daily 
y = rai_eth['close_price']
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(y,marker='.', linestyle='-', linewidth=0.5, label='Hourly')
#change the d to M if you want to see monthly for example 
ax.plot(y.resample('d').mean(),marker='o', markersize=8, linestyle='-', label='Daily Mean Resample')
ax.set_ylabel('Close Price')
ax.set_xlabel('Time')
plt.title('RAI ETH Price',fontsize=20)
ax.legend();

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = rai_eth['close_price'].rolling(window).mean()
mae = mean_absolute_error(rai_eth.close_price[window:], rolling_mean[window:])
deviation = np.std(rai_eth.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
#lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 

In [ ]:
lower_bound[926]

In [ ]:
upper_bound = rolling_mean + (mae + scale * deviation)
#change like you change the lower_bound if you want to see other datapoints or whole thing
upper_bound[926]

In [ ]:
rai_eth['visor_fees'] = rai_eth['fees_estimate_eth'] *.05

In [ ]:
visor_rai_eth_fees = rai_eth.visor_fees.sum()
visor_rai_eth_fees

**Previous 8 Hours**

In [ ]:
plotMovingAverage(rai_eth['close_price'],8, plot_intervals=True) 
plt.title('RAI ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
moving_average(rai_eth['close_price'], 8)

In [ ]:
plotMovingAverage(rai_eth['close_price'],2, plot_intervals=True) 
plt.title('RAI ETH Moving Average Past 1 Hours',fontsize=15)

### MIR-UST

In [ ]:
mir_ust = pd.read_csv("../input/mir-ust/MIR_UST.csv")

In [ ]:
mir_ust.head()

In [ ]:
mir_ust['time'] = pd.to_datetime(mir_ust['timeInterval.minute'], format = '%Y-%m-%d')
mir_ust.index =mir_ust["time"]
mir_ust= mir_ust.loc['2021-03-15':'2021-04-25']

In [ ]:
del mir_ust['Unnamed: 0']

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = mir_ust['close_price'].rolling(window).mean()
mae = mean_absolute_error(mir_ust.close_price[window:], rolling_mean[window:])
deviation = np.std(mir_ust.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
#lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 

In [ ]:
#previous hour lower bound 
lower_bound[854]

In [ ]:
#previous hour upper bound 
upper_bound = rolling_mean + (mae + scale * deviation)
#change like you change the lower_bound if you want to see other datapoints or whole thing
upper_bound[854]

In [ ]:
mir_ust.columns = ['timeInterval.minute', 'baseCurrency.symbol',
       'baseCurrency.address', 'baseAmount', 'quoteCurrency.symbol',
       'quoteCurrency.address', 'quoteAmount', 'trades', 'quotePrice',
       'maximum_price', 'minimum_price', 'open_price', 'close_price',
       'total_fees_estimate_(UST)', 'Cum_fees_estimate_UST', 'twap_7d',
       'time']

In [ ]:
mir_ust['fees_eth'] = mir_ust['total_fees_estimate_(UST)']*0.00036352
mir_ust['visor_fees_eth'] = mir_ust['fees_eth']*.05

In [ ]:
visor_mir_ust_eth_fees = mir_ust['visor_fees_eth'].sum()
visor_mir_ust_eth_fees

In [ ]:
moving_average(mir_ust['close_price'], 8)

**Previous 8 Hours**

In [ ]:
plotMovingAverage(mir_ust['close_price'],8, plot_intervals=True) 
plt.title('MIR UST Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(mir_ust['close_price'],2, plot_intervals=True) 
plt.title('MIR UST Moving Average Past 1 Hours',fontsize=15)

### TRIBE-FEI

In [ ]:
tribe_fei = pd.read_csv("../input/tribe-fei/TRIBE_FEI.csv")
tribe_fei['time'] = pd.to_datetime(tribe_fei['timeInterval.minute'], format = '%Y-%m-%d')
tribe_fei.index =tribe_fei["time"]
tribe_fei= tribe_fei.loc['2021-03-15':'2021-04-25']

In [ ]:
tribe_fei.head()

In [ ]:
del tribe_fei['Unnamed: 0']

In [ ]:
tribe_fei.columns=['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(FEI)',
       'Cum_fees_estimate_FEI', 'twap_7d', 'time']

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = tribe_fei['close_price'].rolling(window).mean()
mae = mean_absolute_error(tribe_fei.close_price[window:], rolling_mean[window:])
deviation = np.std(tribe_fei.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
#lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 

In [ ]:
lower_bound[517]

In [ ]:
#previous hour upper bound 
upper_bound = rolling_mean + (mae + scale * deviation)
#change like you change the lower_bound if you want to see other datapoints or whole thing
upper_bound[517]

In [ ]:
tribe_fei["fees_eth"]=tribe_fei['total_fees_estimate_(FEI)']*0.00034478

In [ ]:
tribe_fei['visor_fees']=tribe_fei['fees_eth']*.05

In [ ]:
visor_fees_tribe_fei = tribe_fei.fees_eth.sum()
visor_fees_tribe_fei

**Previous 8 Hours**

In [ ]:
plotMovingAverage(tribe_fei['close_price'],8, plot_intervals=True) 
plt.title('TRIBE FEI Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(tribe_fei['close_price'],2, plot_intervals=True) 
plt.title('TRIBE FEI Moving Average Past 1 Hours',fontsize=15)

### UNI-ETH

In [ ]:
uni_eth = pd.read_csv("../input/uni-weth/UNI_WETH.csv")

In [ ]:
uni_eth["time"] = pd.to_datetime(uni_eth['timeInterval.minute'], format = '%Y-%m-%d')

In [ ]:
uni_eth.head()

In [ ]:
uni_eth.index =uni_eth["time"]

In [ ]:
uni_eth['fees_eth']=uni_eth['total_fees_estimate_(WETH)']

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = uni_eth['close_price'].rolling(window).mean()
mae = mean_absolute_error(uni_eth.close_price[window:], rolling_mean[window:])
deviation = np.std(uni_eth.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
#lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 
lower_bound[1343]
#change like you change the lower_bound if you want to see other datapoints or whole thing

In [ ]:
#upper bound for the previous hour 
upper_bound = rolling_mean + (mae + scale * deviation)
upper_bound[1343]

In [ ]:
uni_eth['visor_fees']=uni_eth['total_fees_estimate_(WETH)']*.05

In [ ]:
visor_uni_eth_fees = uni_eth['visor_fees'].sum()
visor_uni_eth_fees

In [ ]:
del uni_eth['Unnamed: 0']

In [ ]:
uni_eth.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(WETH)',
       'Cum_fees_estimate_WETH', 'twap_7d', 'time', 'fees_eth', 'visor_fees']

In [ ]:
uni_eth.head()

In [ ]:
plotMovingAverage(uni_eth['close_price'],8, plot_intervals=True) 
plt.title('UNI ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(uni_eth['close_price'],2, plot_intervals=True) 
plt.title('UNI ETH Moving Average Past 1 Hours',fontsize=15)

### DAI-ETH

In [ ]:
dai_eth=pd.read_csv("../input/weth-dai/WETH_DAI.csv")
dai_eth['time'] = pd.to_datetime(dai_eth['timeInterval.minute'], format = '%Y-%m-%d')
dai_eth.index =dai_eth["time"]
dai_eth= dai_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
del dai_eth['Unnamed: 0']

In [ ]:
dai_eth.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate',
       'Cumulative_fees_estimate_DAI', 'twap_7d', 'time']

In [ ]:
#see the last hour set window =2 
#see the last 2 hours set =3 etc 
window=2
#dont change this scale 
scale=1.96
rolling_mean = dai_eth['close_price'].rolling(window).mean()
mae = mean_absolute_error(dai_eth.close_price[window:], rolling_mean[window:])
deviation = np.std(dai_eth.close_price[window:] - rolling_mean[window:])
lower_bound = rolling_mean - (mae + scale * deviation)
lower_bound.shape
#depending on if you want to see the most recent datapoint this is showing the most recent lower bound price in the past hour
#i got that by uncommenting lower_bound.shape and finding out that it was length of 1006 so calling 1005 is the most recent datapoint
#remove the box like this lower_bound and run that to see all data points for lower bound 
#change like you change the lower_bound if you want to see other datapoints or whole thing

In [ ]:
lower_bound[1007]

In [ ]:
#upper bound for the previous hour 
upper_bound = rolling_mean + (mae + scale * deviation)
upper_bound[1007]

In [ ]:
dai_eth['fees_eth']=dai_eth['total_fees_estimate']*0.00036332

In [ ]:
dai_eth['visor_fees']=dai_eth['fees_eth']*.05

In [ ]:
dai_eth_visor_fees = dai_eth['visor_fees'].sum()
dai_eth_visor_fees

In [ ]:
plotMovingAverage(dai_eth['close_price'],8, plot_intervals=True) 
plt.title('DAI ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(dai_eth['close_price'],2, plot_intervals=True) 
plt.title('DAI ETH Moving Average Past 1 Hours',fontsize=15)

### FRAX-USDC

In [ ]:
frax_usdc=pd.read_csv('../input/frax-usdc/USDC_FRAX.csv')

In [ ]:
del frax_usdc['Unnamed: 0']

In [ ]:
frax_usdc.head()

In [ ]:
frax_usdc['time'] = pd.to_datetime(frax_usdc['timeInterval.minute'], format = '%Y-%m-%d')
frax_usdc.index =frax_usdc["time"]
frax_usdc= frax_usdc.loc['2021-03-15':'2021-04-25']

In [ ]:
frax_usdc.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(FRAX)',
       'Cumulative_fees_estimate_FRAX', 'twap_7d', 'time']

In [ ]:
frax_usdc['fees_eth']=frax_usdc['total_fees_estimate_(FRAX)']*0.0003657

In [ ]:
frax_usdc['visor_fees']=frax_usdc['fees_eth']*.05

In [ ]:
frax_usdc_visor_fees = frax_usdc['visor_fees'].sum()

In [ ]:
frax_usdc_visor_fees

In [ ]:
frax_usdc.head()

In [ ]:
plotMovingAverage(frax_usdc['close_price'],8, plot_intervals=True) 
plt.title('FRAX USDC Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(frax_usdc['close_price'],2, plot_intervals=True) 
plt.title('FRAX USDC Moving Average Past 1 Hours',fontsize=15)

### FEI-ETH

In [ ]:
fei_eth=pd.read_csv("../input/weth-fei/WETH_FEI.csv")
fei_eth['time'] = pd.to_datetime(fei_eth['timeInterval.minute'], format = '%Y-%m-%d')
fei_eth.index =fei_eth["time"]
fei_eth= fei_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
del fei_eth['Unnamed: 0']

In [ ]:
fei_eth.columns =['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(FEI)',
       'Cum_fees_estimate_FEI', 'twap_7d', 'time']

In [ ]:
fei_eth['fees_eth']=fei_eth['total_fees_estimate_(FEI)']*0.00034478

In [ ]:
fei_eth.head()

In [ ]:
fei_eth['visor_fees']=fei_eth['fees_eth']*.05

In [ ]:
visor_fei_eth_fees = fei_eth.visor_fees.sum()
visor_fei_eth_fees

In [ ]:
plotMovingAverage(fei_eth['close_price'],8, plot_intervals=True) 
plt.title('FEI ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(fei_eth['close_price'],2, plot_intervals=True) 
plt.title('FEI ETH Moving Average Past 1 Hours',fontsize=15)

### LUSD-ETH

In [ ]:
lusd_eth=pd.read_csv("../input/weth-lusdcsv/WETH_LUSD.csv")
lusd_eth['time'] = pd.to_datetime(lusd_eth['timeInterval.minute'], format = '%Y-%m-%d')

In [ ]:
lusd_eth.index =lusd_eth["time"]

In [ ]:
lusd_eth= lusd_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
del lusd_eth['Unnamed: 0']

In [ ]:
lusd_eth.columns =['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(LUSD)',
       'Cumulative_fees_estimate_LUSD', 'twap_7d', 'time']

In [ ]:
lusd_eth['fees_eth']=lusd_eth['total_fees_estimate_(LUSD)']*0.0003653

In [ ]:
lusd_eth['visor_fees']=lusd_eth.fees_eth*.05

In [ ]:
lusd_eth.head()


In [ ]:
visor_lusd_eth_fees = lusd_eth['visor_fees'].sum()
visor_lusd_eth_fees

In [ ]:
plotMovingAverage(lusd_eth['close_price'],8, plot_intervals=True) 
plt.title('LUSD ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(lusd_eth['close_price'],2, plot_intervals=True) 
plt.title('LUSD ETH Moving Average Past 1 Hours',fontsize=15)

### ETH-USDT

In [ ]:
eth_usdt = pd.read_csv("../input/weth-usdt/WETH_USDT.csv")
eth_usdt['time'] = pd.to_datetime(eth_usdt['timeInterval.minute'], format = '%Y-%m-%d')
eth_usdt.index =eth_usdt["time"]
eth_usdt= eth_usdt.loc['2021-03-15':'2021-04-25']

In [ ]:
eth_usdt.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(USDT)',
       'Cumulative_fees_estimate_USDT', 'twap_7d', 'time', 'visor_fees']

In [ ]:
eth_usdt['fees_eth']=eth_usdt['total_fees_estimate_(USDT)']*0.0003640

In [ ]:
eth_usdt['visor_fees']=eth_usdt['fees_eth']*.05

In [ ]:
visor_eth_usdt_fees = eth_usdt['visor_fees'].sum()
visor_eth_usdt_fees

In [ ]:
eth_usdt.head()

In [ ]:
plotMovingAverage(eth_usdt['close_price'],8, plot_intervals=True) 
plt.title('ETH USDT Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(eth_usdt['close_price'],2, plot_intervals=True) 
plt.title('ETH USDT Moving Average Past 1 Hours',fontsize=15)

### WBTC-ETH

In [ ]:
wbtc_eth = pd.read_csv("../input/weth-wbtc/WETH_WBTC.csv")
wbtc_eth['time'] = pd.to_datetime(wbtc_eth['timeInterval.minute'], format = '%Y-%m-%d')
wbtc_eth.index =wbtc_eth["time"]
wbtc_eth= wbtc_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
del wbtc_eth['Unnamed: 0']

In [ ]:
wbtc_eth.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(WBTC)',
       'Cum_fees_estimate_WBTC', 'twap_7d', 'time']

In [ ]:
wbtc_eth.head()

In [ ]:
wbtc_eth['fees_eth'] = wbtc_eth['total_fees_estimate_(WBTC)']*20.7950872

In [ ]:
wbtc_eth['visor_fees'] = wbtc_eth['fees_eth']*.05
visor_wbtc_eth_fees = wbtc_eth['visor_fees'].sum()
visor_wbtc_eth_fees

In [ ]:
plotMovingAverage(wbtc_eth['close_price'],8, plot_intervals=True) 
plt.title('WBTC ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(wbtc_eth['close_price'],2, plot_intervals=True) 
plt.title('WBTC ETH Moving Average Past 1 Hours',fontsize=15)

### WISE-ETH

In [ ]:
wise_eth = pd.read_csv('../input/wise-weth/WISE_WETH.csv')
wise_eth['time'] = pd.to_datetime(wise_eth['timeInterval.minute'], format = '%Y-%m-%d')
wise_eth.index =wise_eth["time"]
wise_eth= wise_eth.loc['2021-03-15':'2021-04-25']

In [ ]:
del wise_eth['Unnamed: 0']

In [ ]:
wise_eth.columns = ['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(ETH)',
       'Cum_fees_estimate_ETH', 'twap_7d', 'time']

In [ ]:
wise_eth.head()

### Fees in ETH 

In [ ]:
wise_eth['fees']=wise_eth['total_fees_estimate_(ETH)']*2.1364819

In [ ]:
wise_eth['visor_fees']=wise_eth['fees']*.05

In [ ]:
wise_eth['visor_fees'].sum()

In [ ]:
visor_wise_eth_fees = wise_eth['visor_fees'].sum()
visor_wise_eth_fees

In [ ]:
plotMovingAverage(wise_eth['close_price'],8, plot_intervals=True) 
plt.title('WISE ETH Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(wise_eth['close_price'],2, plot_intervals=True) 
plt.title('WISE ETH Moving Average Past 1 Hours',fontsize=15)

### USDT-HKMT

In [ ]:
usdt_hkmt = pd.read_csv("../input/usdt-hkmt/USDT_HKMT.csv")

In [ ]:
del usdt_hkmt['Unnamed: 0']

In [ ]:
usdt_hkmt.columns=['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(HKMT)',
       'Cumulative_fees_estimate_HKMT', 'twap_7d']

In [ ]:
usdt_hkmt['time'] = pd.to_datetime(usdt_hkmt['timeInterval.minute'], format = '%Y-%m-%d')
usdt_hkmt.index =usdt_hkmt["time"]
usdt_hkmt= usdt_hkmt.loc['2021-03-15':'2021-04-25']

In [ ]:
usdt_hkmt['fees_eth']=usdt_hkmt['total_fees_estimate_(HKMT)']*0.0000465

In [ ]:
usdt_hkmt['visor_fees']=usdt_hkmt['fees_eth']*.05

In [ ]:
usdt_hkmt_visor_fees = usdt_hkmt['visor_fees'].sum()

In [ ]:
usdt_hkmt_visor_fees

In [ ]:
plotMovingAverage(usdt_hkmt['close_price'],2, plot_intervals=True) 
plt.title('USDT HKMT Moving Average Past 1 Hours',fontsize=15)

In [ ]:
plotMovingAverage(usdt_hkmt['close_price'],3, plot_intervals=True) 
plt.title('USDT HKMT Moving Average Past 2 Hours',fontsize=15)

## ETH-LINK

In [ ]:
eth_link= pd.read_csv("../input/eth-link/WETH_LINK.csv")

In [ ]:
del eth_link['Unnamed: 0']

In [ ]:
eth_link.columns=['timeInterval.minute', 'baseCurrency.symbol', 'baseCurrency.address',
       'baseAmount', 'quoteCurrency.symbol', 'quoteCurrency.address',
       'quoteAmount', 'trades', 'quotePrice', 'maximum_price', 'minimum_price',
       'open_price', 'close_price', 'total_fees_estimate_(LINK)',
       'Cumulative_fees_estimate_LINK', 'twap_7d']

In [ ]:
eth_link['fees'] = eth_link['total_fees_estimate_(LINK)']*0.0137346

In [ ]:
eth_link_visor_fees = eth_link['fees'].sum()

In [ ]:
plotMovingAverage(eth_link['close_price'],8, plot_intervals=True) 
plt.title('ETH LINK Moving Average Past 8 Hours',fontsize=15)

In [ ]:
plotMovingAverage(eth_link['close_price'],2, plot_intervals=True) 
plt.title('ETH LINK Moving Average Past 1 Hours',fontsize=15)


## Calculating total Visor fees

In [ ]:

visor_prices = pd.read_csv("../input/visor-data/visor_prices.csv")

In [ ]:
visor_prices['dt'] = pd.to_datetime(visor_prices['time'], format = '%Y-%m-%d')
visor_prices.index =visor_prices["dt"]
visor_prices= visor_prices.loc['2021-03-15':'2021-04-25']

In [ ]:
del visor_prices['visor_marketcap']

In [ ]:
visor_prices.head()

In [ ]:
visor_total = visor_mir_ust_eth_fees+visor_rai_eth_fees+visor_fees_usdc_eth+visor_fees_tribe_fei+visor_uni_eth_fees+dai_eth_visor_fees+visor_fei_eth_fees+visor_lusd_eth_fees+visor_eth_usdt_fees+visor_wbtc_eth_fees+visor_wise_eth_fees+frax_usdc_visor_fees+usdc_usdt_visor_fees+usdt_hkmt_visor_fees+eth_link_visor_fees

In [ ]:
visor_total=visor_total *(.1)

### Ethereum to Visor

In [ ]:
ETH_to_VISR = visor_total*855.6147910
ETH_to_VISR

In [ ]:
VISR_to_USD = ETH_to_VISR*3.2388137
VISR_to_USD

In [ ]:
VISR_to_USD/5000000

## Calculating optimal VISR buys (post-local maximum / pre-trend reversal strategy)

In [ ]:
new= pd.read_csv('../input/visornew/visornew.csv')

In [ ]:
new.head()
new['timedt'] = pd.to_datetime(new['dt'], format = '%Y-%m-%d')

In [ ]:
new.head()

In [ ]:
plt.figure(figsize=(20,5))
plt.title('New Visor Price',fontsize=20)
plt.xticks(rotation=45)
sns.lineplot(x='timedt',y='visor_price', data=new, color='deepskyblue')
#lines for buys 

In [ ]:
visor_prices = pd.read_csv("../input/visor-data/visor_prices.csv")

In [ ]:
visor_prices['dt'] = pd.to_datetime(visor_prices['time'], format = '%Y-%m-%d')
visor_prices.index =visor_prices["dt"]
visor_prices= visor_prices.loc['2021-03-15':'2021-04-25']

In [ ]:
del visor_prices['visor_marketcap']

In [ ]:
visor_prices.columns = ['time', 'visor_price', 'visor_total_volume', 'dt']

In [ ]:
visor_prices['time_dt'] = pd.to_datetime(visor_prices['time'], format = '%Y-%m-%d')

In [ ]:
visor_prices.head()

In [ ]:
visor_7d = visor_prices.rolling(7, center=True).mean()
visor_7d.head(10)

In [ ]:
visor_weekly_mean = visor_prices.resample('W').mean()
visor_weekly_mean.tail(3)

### 7 day rolling 

In [ ]:
start, end = '2021-03', '2021-04'

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(visor_prices.loc[start:end, 'visor_price'],
marker='.', linestyle='-', linewidth=0.5, label='daily')
ax.plot(visor_weekly_mean.loc[start:end, 'visor_price'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.plot(visor_7d.loc[start:end, 'visor_price'],
marker='.', linestyle='-', label='7-d Rolling Mean')
plt.title('Visor Price 7 day rolling',fontsize=20)
ax.set_ylabel('Visor Prices')
ax.legend()


In [ ]:
plt.figure(figsize=(20,5))
plt.title('Visor Price no changes',fontsize=20)
ax = visor_prices.loc['2021-03-15':'2021-04-25', 'visor_price'].plot( linestyle='-', markerfacecolor='y',color='r',markersize=7)
ax.set_ylabel('Visor Prices')